In [ ]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
data = pd.read_csv('/Users/macintosh/Desktop/TitleAnalyzer/data/book30-listing-test.csv',encoding = "ISO-8859-1")

In [ ]:
###dsa as as 
columns = ['Id', 'Image', 'Image_link', 'Title', 'Author', 'Class', 'Genre']
data.columns = columns

In [ ]:
books = pd.DataFrame(data['Title'])
author = pd.DataFrame(data['Author'])
genre = pd.DataFrame(data['Genre'])

In [ ]:
data['Author'] = data['Author'].fillna('No Book')
data['Title'] = data['Title'].fillna('No Book')

In [ ]:
print (len(books))
print (len(genre))
genre.head(2)
#books[50000:50110]

5699
5699


,Genre
0,Test Preparation
1,Biographies & Memoirs


In [28]:
print (len(books))
print (len(genre))
#print (len(all_books))
#print (len(all_genres))

5699
5699


In [29]:
genre['Genre'].unique()

array(['Test Preparation', 'Biographies & Memoirs',
       'Religion & Spirituality', 'Arts & Photography',
       'Literature & Fiction', 'Engineering & Transportation',
       'Science & Math', 'Christian Books & Bibles', "Children's Books",
       'Self-Help', 'Law', 'Politics & Social Sciences',
       'Health, Fitness & Dieting', 'Business & Money',
       'Parenting & Relationships', 'Sports & Outdoors',
       'Computers & Technology', 'Cookbooks, Food & Wine',
       'Crafts, Hobbies & Home', 'Travel', 'Humor & Entertainment',
       'Reference', 'Calendars', 'History', 'Comics & Graphic Novels',
       'Teen & Young Adult', 'Romance', 'Science Fiction & Fantasy',
       'Mystery, Thriller & Suspense', 'Medical Books'], dtype=object)

In [30]:
from sklearn.preprocessing import LabelEncoder

feat = ['Genre']
for x in feat:
    le = LabelEncoder()
    le.fit(list(genre[x].values))
    genre[x] = le.transform(list(genre[x]))
    

In [31]:
genre['Genre'].unique()

array([28,  1, 21,  0, 15, 10, 23,  5,  4, 25, 14, 19, 11,  2, 18, 26,  7,
        8,  9, 29, 13, 20,  3, 12,  6, 27, 22, 24, 17, 16])

In [32]:
data['everything'] = pd.DataFrame(data['Title'] + ' ' + data['Author'])
print (data['everything'].head(5))
#for i,j in enumerate(data['everything']):
#    data['everything'][i] = data['everything'][i].lower()
#data['everything'] = data['everything'].str.lower()

0    Barron's GRE, 21st Edition Sharon Weiner Green...
1    George Balanchine: The Ballet Maker (Eminent L...
2    A Partner in Holiness: Deepening Mindfulness, ...
3    Construction Scheduling: Principles and Practi...
4    Literature and Its Writers: A Compact Introduc...
Name: everything, dtype: object


In [33]:
#(data['everything'].head(5))

In [13]:
def change(t):
    t = t.split()
    return ' '.join([(i) for (i) in t if i not in stop])

In [14]:
from nltk.corpus import stopwords
stop = list(stopwords.words('english'))
stop[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [15]:
#remove_list = ['the','at','or']
#for i,j in enumerate(data['everything']):
#    t = data['everything'][i].split()
#    data['everything'][i] = ' '.join([str(i) for (i) in t if i not in stop])

In [16]:
data['Title'].apply(change)

0        Magnets: Pulling Together, Pushing Apart (Amaz...
1        Energy Security (SAGE Library International Se...
2                An Amish Gathering: Life Lancaster County
3        City Rocks Idaho: A Climber's Guide (Regional ...
4        Como vencer el insomnio. Tecnicas, reglas cons...
                               ...                        
51294                                         Extreme Prey
51295    The Crime Writer's Guide Police Practice Proce...
51296                          The Jezebel Remedy: A novel
51297                                              Startup
51298       Killer Takeout: A Key West Food Critic Mystery
Name: Title, Length: 51299, dtype: object

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vectorizer = TfidfVectorizer(min_df=2, max_features=70000, strip_accents='unicode',lowercase =True,
                            analyzer='word', token_pattern=r'\w+', use_idf=True, 
                            smooth_idf=True, sublinear_tf=True, stop_words = 'english')
vectors = vectorizer.fit_transform(data['Title'])
vectors.shape

(51299, 17670)

In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split 

In [20]:
X_train, X_test, y_train, y_test = train_test_split(vectors, genre['Genre'], test_size=5700)

In [21]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(45599, 17670)
(45599,)
(5700, 17670)
(5700,)


In [111]:
type(books)

pandas.core.frame.DataFrame

## Gaussian NB

In [121]:
clf = MultinomialNB(alpha=.45)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print (metrics.f1_score(y_test, pred, average='macro'))
print (metrics.accuracy_score(y_test, pred))

0.5656276484993898
0.5710526315789474


## Logistic Regression

In [132]:
from sklearn import linear_model
clf = linear_model.LogisticRegression(solver= 'sag',max_iter=200,random_state=450)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print (metrics.f1_score(y_test, pred, average='macro'))
print (metrics.accuracy_score(y_test, pred))

0.5692253904423911
0.5670175438596491


In [133]:
text = ['I too had a Love Story']
text[0] = text[0].lower()
#text = list(text)
s = (vectorizer.transform(text))
#s = vectorizer.fit_transform(df)
print (s.shape)
d = (clf.predict(s))
prob = clf.predict_proba(s)
print(prob)

(1, 17670)
[[0.0114935  0.25445727 0.00598315 0.0024435  0.06244526 0.04218315
  0.05048458 0.00323088 0.00150069 0.03032337 0.00560619 0.01294365
  0.01365645 0.01732811 0.00348959 0.07102182 0.00252977 0.00184289
  0.08880621 0.00910954 0.00663015 0.00742377 0.13889971 0.0094223
  0.00687254 0.06822852 0.01841364 0.04879352 0.00136905 0.00306722]]


In [123]:
le.inverse_transform(d)[0]

'Biographies & Memoirs'

## Saving Model

In [125]:
import joblib
joblib.dump(clf, 'best.pkl')
print ("Model Saved")

Model Saved


In [126]:
clf = joblib.load('best.pkl')

In [127]:
clf

LogisticRegression(max_iter=200, random_state=450, solver='sag')

## Neural Network

In [128]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation='logistic', alpha=0.00003, batch_size='auto',
                   beta_1=0.9, beta_2=0.999, early_stopping=False,
                   epsilon=1e-08, hidden_layer_sizes=(20,), learning_rate='constant',
                   learning_rate_init=0.003, max_iter=200, momentum=0.9,
                   nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
                   solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
                   warm_start=False)
clf.fit(X_train, y_train) 
pred = clf.predict(X_test)
print (metrics.f1_score(y_test, pred, average='macro'))
print (metrics.accuracy_score(y_test, pred))

0.4771350150049706
0.47368421052631576


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [84]:
text = ['Until It Fades: A Novel']
#text = list(text)
s = (vectorizer.transform(text))
#s = vectorizer.fit_transform(df)
print (s.shape)
d = (clf.predict(s))

(1, 25000)


In [85]:
le.inverse_transform(d)[0]

'Literature & Fiction'

## SVM

In [80]:
import xgboost as xgb
dtrain = xgb.DMatrix(data=X_train, label = y_train)
dtest = xgb.DMatrix(data=X_test)

In [81]:
params = {
    'objective':'multi:softmax',
    'eval_metric':'mlogloss',
    'eta':0.025,
    'max_depth':10,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'min_child_weight':5,
    'num_class': 32
    
}

In [82]:
bst = xgb.cv(params, dtrain, num_boost_round=100,
early_stopping_rounds=40, nfold=5, verbose_eval=10)

[0]	train-mlogloss:3.42438+0.00144	test-mlogloss:3.42634+0.00220
[10]	train-mlogloss:3.17781+0.00150	test-mlogloss:3.19627+0.00770
[20]	train-mlogloss:3.02809+0.00084	test-mlogloss:3.05975+0.01072
[30]	train-mlogloss:2.91582+0.00182	test-mlogloss:2.96007+0.01221
[40]	train-mlogloss:2.82386+0.00159	test-mlogloss:2.88028+0.01366
[50]	train-mlogloss:2.74479+0.00127	test-mlogloss:2.81264+0.01424
[60]	train-mlogloss:2.67549+0.00152	test-mlogloss:2.75366+0.01425
[70]	train-mlogloss:2.61442+0.00212	test-mlogloss:2.70197+0.01408
[80]	train-mlogloss:2.56027+0.00192	test-mlogloss:2.65663+0.01476
[90]	train-mlogloss:2.51269+0.00230	test-mlogloss:2.61733+0.01476
[99]	train-mlogloss:2.47465+0.00255	test-mlogloss:2.58611+0.01541


In [86]:
bst_train = xgb.train(params, dtrain, num_boost_round=10)

In [87]:
p_test = bst_train.predict(dtest)

In [135]:
print (metrics.f1_score(y_test, p_test, average='macro'))
print (metrics.accuracy_score(y_test, p_test))

0.03167892801056606
0.03385964912280702


In [134]:
text = ['Fifty Shades of Grey']
#text = list(text)
s = (vectorizer.transform(text))
#s = vectorizer.fit_transform(df)
print (s.shape)
p = (bst_train.predict(s))

(1, 17670)


TypeError: ('Expecting data to be a DMatrix object, got: ', <class 'scipy.sparse._csr.csr_matrix'>)

In [94]:
le.inverse_transform(d)[0]

'Literature & Fiction'